<a href="https://colab.research.google.com/github/kiwa-debug/IPC_model/blob/main/successful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Data retrivation

In [1]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 6.4 MB/s eta 0:00:00


In [2]:
import xlsxwriter
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from keras.models import load_model
from openpyxl import load_workbook
import tensorflow_hub as hub
tf_hub_embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
                                        name="universal_sentence_encoder")

In [3]:
def IPC_Self_learning_Model(text):
  train_data=pd.read_excel("ipc_data_1.xlsx")
  X = train_data["Offence"].to_list()
  X = np.array(X)
  train_data_1= train_data['Section']
  train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))

  multilabel = MultiLabelBinarizer()
  y = multilabel.fit_transform(train_data_1)


  # Define feature extractor model using TF Hub layer
  inputs = layers.Input(shape=[], dtype=tf.string)
  pretrained_embedding = tf_hub_embedding_layer(inputs) # tokenize text and create embedding
  x = layers.Dense(128, activation="relu")(pretrained_embedding) # add a fully connected layer on top of the embedding
  # Note: you could add more layers here if you wanted to
  outputs = layers.Dense(len(multilabel.classes_), activation="softmax")(x) # create the output layer
  model_1 = tf.keras.Model(inputs=inputs,
                          outputs=outputs)
  


  # Compile the model
  model_1.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])

  model_1.fit(X,  y,
               epochs=30,
                verbose=0)
  
  text = np.array([text])
  model_1_pred_probs = model_1.predict(text)
  n=2
  indices_2 = (-model_1_pred_probs).argsort()[:n]
  q=[]
  for r in range(0,4):
    q.append(multilabel.classes_[indices_2[0,r]])
  

  return q

In [4]:
def IPC_Check(pred,res):
  g=[]
  for r in res:
    if r in pred:
      g.append(True)
    else:
      g.append(False)
  if False in g:
    u="wrong"
  else:
    u="right"
  return u

In [5]:
def Improve(result,off,sec):
  if result=="wrong":

    sec=str(sec)


    myFileName=r'ipc_data_1.xlsx'
    #load the workbook, and put the sheet into a variable
    wb = load_workbook(filename=myFileName)
    ws = wb['Sheet1']

    #max_row is a sheet function that gets the last row in a sheet.
    newRowLocation = ws.max_row 

    #write to the cell you want, specifying row and column, and value :-)
    ws.cell(column=2,row=newRowLocation, value=sec)
    ws.cell(column=3,row=newRowLocation, value=off)
    wb.save(filename=myFileName)
    wb.close()

###Testing

In [6]:
offence_ ="riot and murder"
section_ = ['146 IPC', '302 IPC']

In [8]:
pred_ipc = IPC_Self_learning_Model(offence_)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


1/1 [==============================] - 0s 347ms/step


In [9]:
pred_ipc

['147 IPC', '148 IPC', '506 IPC', '153 IPC']

In [10]:
result = IPC_Check(pred_ipc, section_)

In [11]:
result

'wrong'

In [12]:
Improve(result, offence_, section_)

In [13]:
pred_ipc = IPC_Self_learning_Model(offence_)

1/1 [==============================] - 0s 432ms/step


In [14]:
pred_ipc

['146 IPC', '302 IPC', '148 IPC', '147 IPC']